In [1]:
import os
import sys
import re
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import Trajectory, select_nodes, scale_configuration, pair_dist_cutoff
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
with open(os.path.join(working, 'stats_samples'+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, 'target_samples'+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)   

In [6]:
# show all available spline knots
stats_all['hyperparams']

{'pair': [2.74, 3.252, 3.804, 4.356, 4.908, 5.46],
 'embed': [2.74, 3.252, 3.804, 4.356, 4.908, 5.46]}

In [7]:
# selecty spline knots
pair_index = [0, 1, 2, 3, 4, 5]
multibody_index = [5]

p_ix = np.array([True if i in pair_index else False for i in range(len(stats_all['hyperparams']['pair']))])
m_ix = np.array([True if i in multibody_index else False for i in range(len(stats_all['hyperparams']['embed']))])
print("pair:", np.array(stats_all['hyperparams']['pair'])[p_ix])
print("multibody:", np.array(stats_all['hyperparams']['embed'])[m_ix])

stats = select_nodes(stats_all, p_ix, m_ix)

pair: [2.74  3.252 3.804 4.356 4.908 5.46 ]
multibody: [5.46]


In [8]:
# read corresponding parameters
with open(os.path.join(working, 'pars_in'+'.pickle'), 'rb') as fi:
    pars_list = pickle.load(fi)
    
pars_dict = pars_list[0]
pars_in = pars_list[0]['embed'] + pars_list[0]['pair']
print(pars_list)
print(pars_in)

[{'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802]}]
[-0.684229349, 0.000152487644, -65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802]


In [9]:
multi_pars = []

multi_pars.append(pars_in)

multi_pars.append([-1.48107607e+00, -2.11281591e-06, -7.75748552e+01,  1.23284902e+02,
       -1.05876828e+02,  2.56371829e+01,  7.21885424e-01, -2.01549825e+00])

multi_pars.append([ 4.21813176e+00,  1.28438711e-02,  7.18447209e+02,  2.79873504e+01,
       -3.64063120e+01,  1.55668343e+01, -3.59671207e+00,  7.18079901e-02])

multi_pars.append([-1.70284536e+00,  1.77584742e-03, -2.03091714e+02,  6.67172987e+02,
       -1.39072153e+03,  7.95275451e+02, -2.44611978e+01,  5.09001525e+00])

multi_pars.append([ 1.49705729e+00,  8.14531512e-03, -8.15988329e+02,  5.87519886e+02,
       -1.01285873e+03,  5.10026120e+02, -1.11686150e+01,  1.51343754e+00])

multi_pars.append([ 1.54864560e+00,  1.06062750e-02, -3.00996729e+02,  3.32437763e+02,
       -3.39714171e+02,  9.93126859e+01, -2.04873166e+01,  1.12932344e+00])

multi_pars.append([ 1.08153442e+00,  2.12340093e-03, -2.83828424e+02,  3.30196594e+02,
       -3.61260411e+02,  1.30278313e+02, -3.12845720e+01,  2.97822976e+00])

multi_pars.append([-1.84530425e+00, -2.18389467e-05, -1.46520729e+03,  1.16790147e+02,
       -1.41528209e+02,  7.82448301e+01,  4.04340173e+00, -2.96638562e+00])

In [10]:
targets['md']['weight'] = 1.0
targets['relax']['weight'] = 20.0
targets['eos_bcc']['weight'] = 1.0
targets['eos_fcc']['weight'] = 0.1

In [11]:
def optimize_EAM_box_mp(pars, targets, stats, n_b=3):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_box), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
    
    return tuple([output[1], output[0]])

In [12]:
def optimize_EAM_mp(pars, targets, stats, n_b=3):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, 0.05, 0), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        
    return tuple([output[1], output[0]])

In [13]:
pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):
    
    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, n_b=len(multi_pars))
    
    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)
  
    # initial ordering
    print('Initial params:', multi_pars)
    
    distances = pool.map(get_sd, multi_pars)
    
    optimal_params = zip(distances, multi_pars)
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(6):
        optimal_params = pool.map(get_opt, m_pars)
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print(f"Iteration {i}, best params: {best_params}")

    print('Final Best params:', best_params)

pool.close()
pool.join()

Initial params: [[-0.684229349, 0.000152487644, -65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802], [-1.48107607, -2.11281591e-06, -77.5748552, 123.284902, -105.876828, 25.6371829, 0.721885424, -2.01549825], [4.21813176, 0.0128438711, 718.447209, 27.9873504, -36.406312, 15.5668343, -3.59671207, 0.0718079901], [-1.70284536, 0.00177584742, -203.091714, 667.172987, -1390.72153, 795.275451, -24.4611978, 5.09001525], [1.49705729, 0.00814531512, -815.988329, 587.519886, -1012.85873, 510.02612, -11.168615, 1.51343754], [1.5486456, 0.010606275, -300.996729, 332.437763, -339.714171, 99.3126859, -20.4873166, 1.12932344], [1.08153442, 0.00212340093, -283.828424, 330.196594, -361.260411, 130.278313, -31.284572, 2.97822976], [-1.84530425, -2.18389467e-05, -1465.20729, 116.790147, -141.528209, 78.2448301, 4.04340173, -2.96638562]]


../../statmechlib/statmechlib/forcefields/eam.py:313: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)
../../statmechlib/statmechlib/forcefields/eam.py:313: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)
../../statmechlib/statmechlib/forcefields/eam.py:313: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)


Best params: [(54.27503194288344, [-1.70284536, 0.00177584742, -203.091714, 667.172987, -1390.72153, 795.275451, -24.4611978, 5.09001525]), (54.240005743875294, [1.49705729, 0.00814531512, -815.988329, 587.519886, -1012.85873, 510.02612, -11.168615, 1.51343754]), (53.652934176378395, [4.21813176, 0.0128438711, 718.447209, 27.9873504, -36.406312, 15.5668343, -3.59671207, 0.0718079901]), (52.76251135140194, [-1.84530425, -2.18389467e-05, -1465.20729, 116.790147, -141.528209, 78.2448301, 4.04340173, -2.96638562]), (31.43827028572503, [1.5486456, 0.010606275, -300.996729, 332.437763, -339.714171, 99.3126859, -20.4873166, 1.12932344]), (25.837989712176896, [-0.684229349, 0.000152487644, -65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802]), (24.219641050996596, [1.08153442, 0.00212340093, -283.828424, 330.196594, -361.260411, 130.278313, -31.284572, 2.97822976]), (22.418794500015203, [-1.48107607, -2.11281591e-06, -77.5748552, 123.284902, -105.876828, 25.6371829, 0.72

../../statmechlib/statmechlib/forcefields/eam.py:313: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)


Iteration 0, best params: [(21.49103674809543, array([-6.24020579e-01,  1.56717189e-03,  2.12914997e+03,  5.29907587e+01,
       -1.83396081e+02,  1.39102895e+02, -6.53209376e+01,  1.30808917e+01])), (12.948137956410331, array([-3.37767555e+00, -2.41504064e-04, -3.44242307e+02,  5.22793187e+02,
       -2.91295935e+02,  1.13261398e+02, -3.09070851e+01,  3.63967294e+00])), (2.5117528035204297, array([ 1.05423357e+01, -1.15917861e-03,  2.34441193e+03,  4.00190251e+00,
       -2.81073246e+01,  2.00710199e+01, -6.86369023e+00, -2.35523726e-02])), (1.8825081787049196, array([ 4.52460644e+00,  3.05226136e-03, -2.89542230e+02, -3.04754319e+01,
       -1.18106235e+00,  6.57574358e+00, -1.41873562e+01,  3.48402045e+00])), (0.7229008781749824, array([-9.66863361e-01,  4.36568187e-04,  2.70931443e+02, -2.72593685e+01,
       -6.80560615e+00,  1.94129659e+01, -1.71842182e+01,  4.79856519e+00])), (0.3640499760233485, array([-1.17116738e+00, -7.28802551e-04,  9.59493801e+01,  7.03810758e+00,
        

In [14]:
len(stats[0]), type(stats[0]), len(stats[0]['energy'][0])

KeyError: 0

In [ ]:
def optimize_EAM_per_atom(targets, stats, ieam, multi_pars, n_b=3):
    
    optimal_parameters = []
    
    for i, pars in enumerate(multi_pars):
        sd_ini = sd2_loss(list(pars), stats, targets, utot_EAM_per_box, None, [ieam])
        print(i,' sd2: ini', sd_ini, end='')
        output = fmin(sd2_loss, pars, args=(stats, targets, utot_EAM_per_box, None, [ieam]), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        optimal_parameters.append(tuple([output[1], output[0]]))
        print('sd2: final', output[1])

    n_best = min(n_b, len(optimal_parameters))
    #print(optimal_parameters)
                 
    best_params = sorted(optimal_parameters, key=lambda param: param[0])[:n_best]
    
    print('Best params:', best_params)

    return best_params

In [ ]:
for ieam in range(4, len(stats_opts[it]['hyperparams'])):
    print('ieam:', ieam)
    m_pars = multi_pars
    for i in range(6):
        best_pars = optimize_EAM(targets, stats, ieam, m_pars, n_b=len(m_pars))
        m_pars = [p[1] for p in best_pars]

In [ ]:
optimal_parameters = [(4.407570269501649e-05, [-1.21658318e+00,  3.24863580e-05, -4.73661687e+01,  2.29965951e+01,
       -3.42248343e+00,  3.73956980e+00, -1.39292737e+00, -8.33458565e-01,
        9.29769431e-01, -1.83568989e-01]), (6.17447812420396e-05, [-1.30551487e+00,  1.41920657e-05, -4.61160967e+01,  2.29352328e+01,
       -3.42360958e+00,  3.82313395e+00, -1.54715703e+00, -6.97925842e-01,
        8.68730239e-01, -1.54592379e-01]), (9.723561709685708e-05, [-1.34624502e+00,  7.49457487e-06, -4.58871674e+01,  2.24064873e+01,
       -3.46516902e+00,  4.02263760e+00, -1.89661680e+00, -4.31617423e-01,
        8.06406324e-01, -1.37562101e-01]), (0.00014123622250142564, [-1.34136558e+00,  1.01769934e-05, -4.93998032e+01,  2.21279115e+01,
       -3.50791272e+00,  4.16822104e+00, -2.13414308e+00, -2.66237999e-01,
        7.83819577e-01, -1.36703203e-01])]

In [ ]:
ieam6 = (0.004639595573920272, np.array([ 2.61411686e+00,  1.44689461e-02, -1.90339171e+02,  1.60713127e+02,
       -1.50065201e+02,  3.41733929e+01, -9.38215096e+00, -2.75667517e-01,
       -3.45258425e-01,  3.18322256e-01, -2.84002515e-01, -6.97804133e-02]))

In [ ]:
ieam9 = (0.008696488352891394, np.array([-1.47456565e+00, -4.79869485e-07, -1.68796012e+02,  1.01838014e+02,
       -1.89387663e+02,  1.23197119e+02,  1.67559787e+00, -1.97672693e+00,
        1.48460087e+00, -3.21160959e-01, -9.99990134e-01,  5.45243666e-01]))

In [ ]:
ieam6 = (0.012529698209073237, np.array([ 1.77179483e+00,  1.05192096e-02, -2.93005591e+02,  3.18328136e+02,
       -3.22036226e+02,  9.25544422e+01, -1.89150669e+01,  9.60458059e-01,
        9.09101177e-02, -2.93381125e-01,  1.16059169e-01, -1.21580547e-01]))

In [ ]:
ieam9 = (0.011166762673644966, np.array([-1.43897524e+00,  3.68357996e-06, -4.10835857e+01,  1.01633094e+02,
       -9.73045925e+01,  3.38437384e+01, -3.58251075e+00, -1.67919252e+00,
        8.65702805e-01,  2.65888291e-01, -1.20777286e+00,  4.75855652e-01]))

In [ ]:
ieam6 = (0.004926680089210157, np.array([-6.25140612e-01,  5.84924686e-05,  1.50463356e+01, -2.45628080e+00,
        2.75467959e+00, -1.82521768e+00,  5.82623873e-01, -8.10349686e-02]))

In [ ]:
optimal_parameters = [ieam6]

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['relax']['xyz']])[:-2]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[1][:-2])
        print('Model:', opti_out[1][:-2])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[1][:-2]))/nat)
        plt.plot(np.array(targ_out[1][:-2])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    print('Model:', opti_out[1][:-2])

    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[1][:-2]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_bcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[2][:])
        print('Model:', opti_out[2][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[2][:-1]))/nat)
        plt.plot(np.array(targ_out[2][:-1])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[2][:-1]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_fcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-3][:])
        print('Model:', opti_out[-3][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[3][:-1]))/nat)
        plt.plot(np.array(targ_out[3][:-1])[[2, 3, 0, 1, 4, 5, 6, 7]]/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[3][:-1]])[[2, 3, 0, 1, 4, 5, 6, 7]]/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_fcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-1][:])
        print('Model:', opti_out[-1][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-1][:-1]))/nat)
        plt.plot(np.array(targ_out[-1][:-1])[[2, 3, 0, 1, 4, 5, 6, 7, 8]]/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot( np.array([o for o in opti_out[-1][:-1]])[[2, 3, 0, 1, 4, 5, 6, 7, 8]]/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['md']['xyz']])[:]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box,[5])
    #plt.title('Selected configurations at 0K')
    if i == 0:
        #print('Targ:', targ_out[0][:])
        #print('Model:', opti_out[0][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[0][:]))/nat)
        #plt.plot(np.array(targ_out[0][:])/nat, 'o', label='target'+str(i), markersize=10)
        plt.plot(np.array(targ_out[0][:]), 'o', label='target'+str(i), markersize=5)

        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 1:# and i != 12:
    #    continue
    print(i, params_uopt[0])

    #print('Opti:', [o for o in opti_out[-1][:-1]])
    #plt.plot(np.array([o for o in opti_out[0][:]])/nat, 'x--', label='optimized '+str(i))
    plt.plot(np.array([o for o in opti_out[0][:]]), '--', label='optimized '+str(i), lw=1.0)

#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]', fontsize=30)
plt.xlabel('configuation #', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.533)
#plt.ylim(-8.9, -8.35)

#plt.ylim(-1075,-1035)
#plt.ylim(-1000,-900)
#plt.ylim(-950,-900)
#plt.ylim(-1136,-1133)
plt.ylim(-1180,-800)
plt.xlim(0,820)
#plt.xlim(847,990)

#plt.ylim(-1180,-800)

plt.legend(fontsize=30)
#plt.savefig(plt.savefig(os.path.join(working, 'bcc_300.png')))

In [ ]:
for i, params_uopt in enumerate(optimal_parameters):
    if params_uopt[0] > 1.1e-3:
        continue
    print(i, params_uopt[0], '\n', params_uopt[1])

In [ ]:
stats_opts[it]['hyperparams']

In [ ]:
pars_out = {}
pars_out['pair_nodes'] = stats_opts[it]['hyperparams']
pars_out['mb_node'] = 9
pars_out['params'] = [optimal_parameters[0]]

In [ ]:
optimal_list = []

#with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
#    optimal_list = pickle.load(fi)

#if type(optimal_list) != list:
#    optimal_list = [optimal_list]

optimal_list.append(pars_out)

with open(os.path.join(working, 'pars_out.pickle'), 'wb') as fo:
    pickle.dump(optimal_list, fo)

In [ ]:
with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
    optimal_list = pickle.load(fi)

In [ ]:
optimal_list

In [ ]:
# Total pair potential
pos = targ_ready['relax']['xyz'][0]
bx = targ_ready['relax']['box'][0]
rr, rx = pair_dist_cutoff(pos, bx, 5.5)

In [ ]:
# embedding function
F = lambda d, a: a[0]*d**0.5 + a[1]*d + a[2]*d**2

V = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

# Cohesive potential (cubic splines - same form as V)
phi = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

rho = lambda r, aa, kk: sum([a*(rk - r)**3 for a, rk in zip(aa, kk) if r < rk and r > 0.01])

In [ ]:
optimal_list[-1]['params'][0][1]

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y, V_x_r, rho_x_r

In [ ]:
ene = []
for a in np.linspace(0.5, 2.0, 101):
    z = a*rr
    u_pair = 0.5*np.sum([V(x, pair_y, V_x_r) for x in z.flatten() if x > 0.1])

    u_many = 0.0
    for i in range(z.shape[0]):
        d = np.sum([phi(x, rho_x_a, rho_x_r) for x in z[i] if x > 0.1])
        u_many += F(d, many_y)

    ene.append([a, u_pair, u_many, u_pair+u_many])

ene = np.array(ene)

In [ ]:
#plt.plot(ene[:,0], ene[:,1]/54, label='pair')
#plt.plot(ene[:,0], ene[:,2]/54, label='manybody')
plt.title('Equation of state (T=0K)')
plt.plot(ene[:,0]*2.74, [universal_eos(x, 'W') for x in ene[:,0]], label='universal EOS')
plt.plot(ene[:,0]*2.74, ene[:,3]/128, label='u-total')
plt.hlines(0.0,0.5,6.0,lw=1)
plt.legend()
plt.xlim(2.0, 5.75)
plt.xlim(2.6, 2.85)
plt.ylim(-10,10)
plt.ylim(-9.,-8.5)

plt.xlabel('x')
plt.ylabel(r'U')

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y

In [ ]:
print(V_x_r)
print((0.011166762673644966, np.array([-1.43897524e+00,  3.68357996e-06, -4.10835857e+01,  1.01633094e+02,
       -9.73045925e+01,  3.38437384e+01, -3.58251075e+00, -1.67919252e+00,
        8.65702805e-01,  2.65888291e-01, -1.20777286e+00,  4.75855652e-01])))

In [ ]:
pair_a = optimal_list[-1]['params'][0][1][2:]#pars_in1[2:]
pair_r = optimal_list[-1]['pair_nodes']

rho_a = [1.0]
rho_r = [pair_r[optimal_list[-1]['mb_node']]]

par = optimal_list[-1]['params'][0][1][:2]
many_a = [par[0], 0.0, par[1]]

In [ ]:
pair_a, pair_r, rho_r, many_a, par

In [ ]:
# Ranges of densities and separations

nr = 5000
rcut = pair_r[-1]#stats_opt['hyperparams'][-1]
dr = rcut/nr

rhomax = 12.0*phi(1.5, rho_a, rho_r)
print('rhomax:', rhomax)
rhomax = 1000.0
nrho = 5000
drho = rhomax/nrho

ri = 1.0
ro = 2.0

# W properties
na = 74
ma = 183.84
lc = 3.1653
alat = 'bcc'

# System composition
n_el = 1
el_name = ['W']

In [ ]:
def u_trans(r, unucl, ueq, ri=1.0, ro=2.0):
    """Transition between the core and equillibrium potentials"""
    if r < ri:
        return unucl
    elif r < ro:
        x = (ro + ri - 2*r)/(ro - ri)
        eta = 3/16*x**5 - 5/8*x**3 + 15/16*x + 1/2
        return ueq + eta*(unucl - ueq)
    else:
        return ueq

In [ ]:
# prepare lists of values
dens = np.linspace(0., rhomax-drho, nrho, endpoint=True)
r = np.linspace(0., rcut-dr, nr, endpoint=True)
r2 = np.linspace(0., rcut, 201)

fembd = [F(d, many_a) for d in dens] # W
edens = np.array([phi(x, rho_a, rho_r) for x in r])

# 1. Cubic spline for r > r_o
f_outer = [V(x, pair_a, pair_r) for i, x in enumerate(r)]
# 2. Repulsive core for r < r_i (precalculate up to r_o)
f_inner = [0.0] + [u_core(x) for x in r[1:]]
# 3. Transition region for r_i < r < r_o
fpair = [x*u_trans(x, f_inner[i], f_outer[i]) for i, x in enumerate(r)]
#fpair = f_outer

In [ ]:
# LAMMPS format
pot_path = '../sim/potentials'

with open(os.path.join(pot_path, 'W_lv_bccfcc.eam.fs'), 'w') as fo:
    
    # Comment section
    fo.write('Comment 1\n')
    fo.write('Comment 2\n')
    fo.write('Comment 3\n')
    
    # Number of elements
    fo.write(f"{n_el:5d} ")
    for i in range(n_el):
        fo.write(f"{el_name[i]:2}")
    fo.write("\n")
    
    # number of grid points, step size, and cutoff
    fo.write(f"{nrho:5d}{drho:24.16e}{nr:5d}{dr:24.16e}{rcut:24.16e}\n")
    
    # atomic number, mass, lattice size, lattice type
    fo.write(f"{na:5d} {ma:14.4f} {lc:14.4f} {alat:10}\n")
    
    # Embeding function
    for i in range(nrho//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fembd[i*4:i*4+4]))
    
    # Electronic density
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*edens[i*4:i*4+4]))
        
    # Pair potential
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fpair[i*4:i*4+4]))